# Check Workspace and Import Packages

In [ ]:
import os
import subprocess
import numpy as np
import pandas as pd
import pandas
from datetime import datetime
now = datetime.now()

In [ ]:
# Get the bucket name
my_bucket = os.getenv('WORKSPACE_BUCKET')

# List objects in the bucket
print(subprocess.check_output(f"gsutil ls -r {my_bucket}", shell=True).decode('utf-8'))

----

Create function to set seeds for reproducibility

----

In [ ]:
def set_seeds():
    import os
    import random
    import numpy as np
    import tensorflow as tf

    # Set the seed for numpy
    np.random.seed(42)

    # Set the seed for the Python random module
    random.seed(42)

    # Set the seed for TensorFlow
    tf.random.set_seed(42)

    # Ensure reproducibility with certain environment variables
    os.environ['PYTHONHASHSEED'] = '42'


    ### Hold off on more extensive seeds (below) until verified necessary


    # # Configure TensorFlow to use a single thread if required
    # tf.config.threading.set_intra_op_parallelism_threads(1)
    # tf.config.threading.set_inter_op_parallelism_threads(1)

    # # Optionally, set environment variables to control NumPy threading behavior
    # os.environ['OMP_NUM_THREADS'] = '1'
    # os.environ['MKL_NUM_THREADS'] = '1'

    # # Example to demonstrate reproducibility
    # print("Numpy Random:", np.random.rand(3))
    # print("Python Random:", random.random())

    # # TensorFlow example
    # tf_example = tf.random.uniform([3])
    # print("TensorFlow Random:", tf_example)

    # # PyTorch Example (if using PyTorch)
    # import torch

    # torch.manual_seed(42)
    # if torch.cuda.is_available():
    #     torch.cuda.manual_seed(42)
    #     torch.cuda.manual_seed_all(42)  # if using multi-GPU.
    #     torch.backends.cudnn.deterministic = True  # cuDNN
    #     torch.backends.cudnn.benchmark = False

    # # Generate reproducible random numbers with PyTorch
    # print("PyTorch Random:", torch.rand(3))

----

Create function to start/stop logging RAM usage to file

----

In [ ]:
import os
import psutil
import threading
import time
from google.cloud import storage

def log_memory_usage(stop_event, file_name):
    with open(file_name, 'w') as f:
        while not stop_event.is_set():
            # Log memory usage to a local file
            memory_info = psutil.virtual_memory()
            gb_used = memory_info.used / (1024 ** 3)
            mem_usage = f"{time.ctime()}: {gb_used:.2f} GB\n"
            print(mem_usage)
            f.write(mem_usage)
            f.flush()
            
            # Upload the local file to GCS
            try:
                destination_blob_name = f'logs/{file_name}'
                upload_to_gcs(file_name, destination_blob_name)
            except Exception as e:
                print(f"Failed to upload to GCS: {e}")
                
            time.sleep(30)
            
def upload_to_gcs(source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    # Get the bucket name
    my_bucket = os.getenv('WORKSPACE_BUCKET')
    # Initialize a storage client
    storage_client = storage.Client()
    bucket = storage_client.bucket(my_bucket[5:])
    blob = bucket.blob(destination_blob_name)

    # Upload the file
    blob.upload_from_filename(source_file_name)

#     print(f"File {source_file_name} uploaded to {destination_blob_name}.")

In [ ]:
stop_event = threading.Event()
memory_thread = None
thread_lock = threading.Lock()  # To ensure thread-safe operations

def RAM_start():
    global stop_event
    global memory_thread

    with thread_lock:
        # Clear the stop event if it is set
        if stop_event.is_set():
            stop_event.clear()

        file_name = 'memory_usage.txt'
        
        # Stop the existing thread if it is running
        if memory_thread and memory_thread.is_alive():
            RAM_stop()
        
        # Create and start a new memory logging thread
        memory_thread = threading.Thread(target=log_memory_usage, args=(stop_event, file_name))
        memory_thread.start()
        print("Memory logging started")

def RAM_stop():
    global stop_event
    global memory_thread

    with thread_lock:
        # Set the stop event to signal the thread to stop
        stop_event.set()

        # Wait for the thread to finish if it exists
        if memory_thread:
            memory_thread.join()
            memory_thread = None  # Reset the thread to None
            print("Memory logging stopped")

In [ ]:
RAM_start()

# Data import

## Anxiety Data

### Physical Activity

In [ ]:
# This query represents dataset "AllFitbitSum+Anxiety" for domain "fitbit_activity" and was generated for All of Us Registered Tier Dataset v7
dataset_29237714_fitbit_activity_sql = """
    SELECT
        activity_summary.person_id,
        activity_summary.date,
        activity_summary.activity_calories,
        activity_summary.calories_bmr,
        activity_summary.calories_out,
        activity_summary.elevation,
        activity_summary.fairly_active_minutes,
        activity_summary.floors,
        activity_summary.lightly_active_minutes,
        activity_summary.marginal_calories,
        activity_summary.sedentary_minutes,
        activity_summary.steps,
        activity_summary.very_active_minutes 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.activity_summary` activity_summary   
    WHERE
        activity_summary.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_fitbit = 1 ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (442077)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )"""

dataset_29237714_fitbit_activity_df = pandas.read_gbq(
    dataset_29237714_fitbit_activity_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

### Heart Rate

In [ ]:
# # This query represents dataset "AllFitbitSum+Anxiety" for domain "fitbit_heart_rate_summary" and was generated for All of Us Registered Tier Dataset v7
# dataset_29237714_fitbit_heart_rate_summary_sql = """
#     SELECT
#         heart_rate_summary.person_id,
#         heart_rate_summary.date,
#         heart_rate_summary.zone_name,
#         heart_rate_summary.min_heart_rate,
#         heart_rate_summary.max_heart_rate,
#         heart_rate_summary.minute_in_zone,
#         heart_rate_summary.calorie_count 
#     FROM
#         `""" + os.environ["WORKSPACE_CDR"] + """.heart_rate_summary` heart_rate_summary   
#     WHERE
#         heart_rate_summary.PERSON_ID IN (SELECT
#             distinct person_id  
#         FROM
#             `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
#         WHERE
#             cb_search_person.person_id IN (SELECT
#                 person_id 
#             FROM
#                 `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
#             WHERE
#                 has_fitbit = 1 ) 
#             AND cb_search_person.person_id IN (SELECT
#                 criteria.person_id 
#             FROM
#                 (SELECT
#                     DISTINCT person_id, entry_date, concept_id 
#                 FROM
#                     `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
#                 WHERE
#                     (concept_id IN(SELECT
#                         DISTINCT c.concept_id 
#                     FROM
#                         `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
#                     JOIN
#                         (SELECT
#                             CAST(cr.id as string) AS id       
#                         FROM
#                             `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
#                         WHERE
#                             concept_id IN (442077)       
#                             AND full_text LIKE '%_rank1]%'      ) a 
#                             ON (c.path LIKE CONCAT('%.', a.id, '.%') 
#                             OR c.path LIKE CONCAT('%.', a.id) 
#                             OR c.path LIKE CONCAT(a.id, '.%') 
#                             OR c.path = a.id) 
#                     WHERE
#                         is_standard = 1 
#                         AND is_selectable = 1) 
#                     AND is_standard = 1 )) criteria ) )"""

# dataset_29237714_fitbit_heart_rate_summary_df = pandas.read_gbq(
#     dataset_29237714_fitbit_heart_rate_summary_sql,
#     dialect="standard",
#     use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
#     progress_bar_type="tqdm_notebook")

### Sleep

In [ ]:
# This query represents dataset "AllFitbitSum+Anxiety" for domain "fitbit_sleep_daily_summary" and was generated for All of Us Registered Tier Dataset v7
dataset_29237714_fitbit_sleep_daily_summary_sql = """
    SELECT
        sleep_daily_summary.person_id,
        sleep_daily_summary.sleep_date,
        sleep_daily_summary.is_main_sleep,
        sleep_daily_summary.minute_in_bed,
        sleep_daily_summary.minute_asleep,
        sleep_daily_summary.minute_after_wakeup,
        sleep_daily_summary.minute_awake,
        sleep_daily_summary.minute_restless,
        sleep_daily_summary.minute_deep,
        sleep_daily_summary.minute_light,
        sleep_daily_summary.minute_rem,
        sleep_daily_summary.minute_wake 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.sleep_daily_summary` sleep_daily_summary   
    WHERE
        PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_fitbit = 1 ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (442077)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )"""

dataset_29237714_fitbit_sleep_daily_summary_df = pandas.read_gbq(
    dataset_29237714_fitbit_sleep_daily_summary_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

## Depression

### Physical Activity

In [ ]:
# This query represents dataset "AllFitbitSummaries+Depression" for domain "fitbit_activity" and was generated for All of Us Registered Tier Dataset v7
dataset_78185922_fitbit_activity_sql = """
    SELECT
        activity_summary.person_id,
        activity_summary.date,
        activity_summary.activity_calories,
        activity_summary.calories_bmr,
        activity_summary.calories_out,
        activity_summary.elevation,
        activity_summary.fairly_active_minutes,
        activity_summary.floors,
        activity_summary.lightly_active_minutes,
        activity_summary.marginal_calories,
        activity_summary.sedentary_minutes,
        activity_summary.steps,
        activity_summary.very_active_minutes 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.activity_summary` activity_summary   
    WHERE
        activity_summary.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_fitbit = 1 ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (440383)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )"""

dataset_78185922_fitbit_activity_df = pandas.read_gbq(
    dataset_78185922_fitbit_activity_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

### Heart rate

In [ ]:
# # This query represents dataset "AllFitbitSummaries+Depression" for domain "fitbit_heart_rate_summary" and was generated for All of Us Registered Tier Dataset v7
# dataset_78185922_fitbit_heart_rate_summary_sql = """
#     SELECT
#         heart_rate_summary.person_id,
#         heart_rate_summary.date,
#         heart_rate_summary.zone_name,
#         heart_rate_summary.min_heart_rate,
#         heart_rate_summary.max_heart_rate,
#         heart_rate_summary.minute_in_zone,
#         heart_rate_summary.calorie_count 
#     FROM
#         `""" + os.environ["WORKSPACE_CDR"] + """.heart_rate_summary` heart_rate_summary   
#     WHERE
#         heart_rate_summary.PERSON_ID IN (SELECT
#             distinct person_id  
#         FROM
#             `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
#         WHERE
#             cb_search_person.person_id IN (SELECT
#                 person_id 
#             FROM
#                 `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
#             WHERE
#                 has_fitbit = 1 ) 
#             AND cb_search_person.person_id IN (SELECT
#                 criteria.person_id 
#             FROM
#                 (SELECT
#                     DISTINCT person_id, entry_date, concept_id 
#                 FROM
#                     `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
#                 WHERE
#                     (concept_id IN(SELECT
#                         DISTINCT c.concept_id 
#                     FROM
#                         `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
#                     JOIN
#                         (SELECT
#                             CAST(cr.id as string) AS id       
#                         FROM
#                             `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
#                         WHERE
#                             concept_id IN (440383)       
#                             AND full_text LIKE '%_rank1]%'      ) a 
#                             ON (c.path LIKE CONCAT('%.', a.id, '.%') 
#                             OR c.path LIKE CONCAT('%.', a.id) 
#                             OR c.path LIKE CONCAT(a.id, '.%') 
#                             OR c.path = a.id) 
#                     WHERE
#                         is_standard = 1 
#                         AND is_selectable = 1) 
#                     AND is_standard = 1 )) criteria ) )"""

# dataset_78185922_fitbit_heart_rate_summary_df = pandas.read_gbq(
#     dataset_78185922_fitbit_heart_rate_summary_sql,
#     dialect="standard",
#     use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
#     progress_bar_type="tqdm_notebook")

### Sleep

In [ ]:
# This query represents dataset "AllFitbitSummaries+Depression" for domain "fitbit_sleep_daily_summary" and was generated for All of Us Registered Tier Dataset v7
dataset_78185922_fitbit_sleep_daily_summary_sql = """
    SELECT
        sleep_daily_summary.person_id,
        sleep_daily_summary.sleep_date,
        sleep_daily_summary.is_main_sleep,
        sleep_daily_summary.minute_in_bed,
        sleep_daily_summary.minute_asleep,
        sleep_daily_summary.minute_after_wakeup,
        sleep_daily_summary.minute_awake,
        sleep_daily_summary.minute_restless,
        sleep_daily_summary.minute_deep,
        sleep_daily_summary.minute_light,
        sleep_daily_summary.minute_rem,
        sleep_daily_summary.minute_wake 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.sleep_daily_summary` sleep_daily_summary   
    WHERE
        PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_fitbit = 1 ) 
            AND cb_search_person.person_id IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (440383)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )"""

dataset_78185922_fitbit_sleep_daily_summary_df = pandas.read_gbq(
    dataset_78185922_fitbit_sleep_daily_summary_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

## No Disorder

### Physical Activity

In [ ]:
# This query represents dataset "AllFitbitSum-AnxDep" for domain "fitbit_activity" and was generated for All of Us Registered Tier Dataset v7
dataset_22083694_fitbit_activity_sql = """
    SELECT
        activity_summary.person_id,
        activity_summary.date,
        activity_summary.activity_calories,
        activity_summary.calories_bmr,
        activity_summary.calories_out,
        activity_summary.elevation,
        activity_summary.fairly_active_minutes,
        activity_summary.floors,
        activity_summary.lightly_active_minutes,
        activity_summary.marginal_calories,
        activity_summary.sedentary_minutes,
        activity_summary.steps,
        activity_summary.very_active_minutes 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.activity_summary` activity_summary   
    WHERE
        activity_summary.PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_fitbit = 1 ) 
            AND cb_search_person.person_id NOT IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (440383, 442077)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )"""

dataset_22083694_fitbit_activity_df = pandas.read_gbq(
    dataset_22083694_fitbit_activity_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

### Heart Rate

In [ ]:
# # This query represents dataset "AllFitbitSum-AnxDep" for domain "fitbit_heart_rate_summary" and was generated for All of Us Registered Tier Dataset v7
# dataset_22083694_fitbit_heart_rate_summary_sql = """
#     SELECT
#         heart_rate_summary.person_id,
#         heart_rate_summary.date,
#         heart_rate_summary.zone_name,
#         heart_rate_summary.min_heart_rate,
#         heart_rate_summary.max_heart_rate,
#         heart_rate_summary.minute_in_zone,
#         heart_rate_summary.calorie_count 
#     FROM
#         `""" + os.environ["WORKSPACE_CDR"] + """.heart_rate_summary` heart_rate_summary   
#     WHERE
#         heart_rate_summary.PERSON_ID IN (SELECT
#             distinct person_id  
#         FROM
#             `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
#         WHERE
#             cb_search_person.person_id IN (SELECT
#                 person_id 
#             FROM
#                 `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
#             WHERE
#                 has_fitbit = 1 ) 
#             AND cb_search_person.person_id NOT IN (SELECT
#                 criteria.person_id 
#             FROM
#                 (SELECT
#                     DISTINCT person_id, entry_date, concept_id 
#                 FROM
#                     `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
#                 WHERE
#                     (concept_id IN(SELECT
#                         DISTINCT c.concept_id 
#                     FROM
#                         `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
#                     JOIN
#                         (SELECT
#                             CAST(cr.id as string) AS id       
#                         FROM
#                             `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
#                         WHERE
#                             concept_id IN (440383, 442077)       
#                             AND full_text LIKE '%_rank1]%'      ) a 
#                             ON (c.path LIKE CONCAT('%.', a.id, '.%') 
#                             OR c.path LIKE CONCAT('%.', a.id) 
#                             OR c.path LIKE CONCAT(a.id, '.%') 
#                             OR c.path = a.id) 
#                     WHERE
#                         is_standard = 1 
#                         AND is_selectable = 1) 
#                     AND is_standard = 1 )) criteria ) )"""

# dataset_22083694_fitbit_heart_rate_summary_df = pandas.read_gbq(
#     dataset_22083694_fitbit_heart_rate_summary_sql,
#     dialect="standard",
#     use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
#     progress_bar_type="tqdm_notebook")

# dataset_22083694_fitbit_heart_rate_summary_df.head(5)

### Sleep

In [ ]:
# This query represents dataset "AllFitbitSum-AnxDep" for domain "fitbit_sleep_daily_summary" and was generated for All of Us Registered Tier Dataset v7
dataset_22083694_fitbit_sleep_daily_summary_sql = """
    SELECT
        sleep_daily_summary.person_id,
        sleep_daily_summary.sleep_date,
        sleep_daily_summary.is_main_sleep,
        sleep_daily_summary.minute_in_bed,
        sleep_daily_summary.minute_asleep,
        sleep_daily_summary.minute_after_wakeup,
        sleep_daily_summary.minute_awake,
        sleep_daily_summary.minute_restless,
        sleep_daily_summary.minute_deep,
        sleep_daily_summary.minute_light,
        sleep_daily_summary.minute_rem,
        sleep_daily_summary.minute_wake 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.sleep_daily_summary` sleep_daily_summary   
    WHERE
        PERSON_ID IN (SELECT
            distinct person_id  
        FROM
            `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person  
        WHERE
            cb_search_person.person_id IN (SELECT
                person_id 
            FROM
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p 
            WHERE
                has_fitbit = 1 ) 
            AND cb_search_person.person_id NOT IN (SELECT
                criteria.person_id 
            FROM
                (SELECT
                    DISTINCT person_id, entry_date, concept_id 
                FROM
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                WHERE
                    (concept_id IN(SELECT
                        DISTINCT c.concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                    JOIN
                        (SELECT
                            CAST(cr.id as string) AS id       
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` cr       
                        WHERE
                            concept_id IN (440383, 442077)       
                            AND full_text LIKE '%_rank1]%'      ) a 
                            ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                            OR c.path LIKE CONCAT('%.', a.id) 
                            OR c.path LIKE CONCAT(a.id, '.%') 
                            OR c.path = a.id) 
                    WHERE
                        is_standard = 1 
                        AND is_selectable = 1) 
                    AND is_standard = 1 )) criteria ) )"""

dataset_22083694_fitbit_sleep_daily_summary_df = pandas.read_gbq(
    dataset_22083694_fitbit_sleep_daily_summary_sql,
    dialect="standard",
    use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
    progress_bar_type="tqdm_notebook")

dataset_22083694_fitbit_sleep_daily_summary_df.head(5)

# Demographics & Heart Rate Level 

  
---  

__For each Cohort:__

- Anxiety
- Depression
- No Anxiety AND No Depression

## Define queries for different cohorts

These queries represent datasets

- "Fitbit_SleepActSum_HRlevel+Depression"
- "Fitbit_SleepActSum_HRlevel+Anxiety"
- "Fitbit_SleepActSum_HRlevel-AnxDep"

for domain 

- "fitbit_heart_rate_level" 
- "person"

and was generated for All of Us Registered Tier Dataset v7

In [ ]:
import os
import pandas as pd

# Define queries for different cohorts
cohorts = {
    'depression': 440383,
    'anxiety': 442077,
    'not_anxiety_depression': [440383, 442077]
}

### Base SQL template - HR

In [ ]:
# Base SQL template - HR
sql_template_hr = """
    SELECT
        heart_rate_minute_level.person_id,
        CAST(heart_rate_minute_level.datetime AS DATE) as date,
        AVG(heart_rate_value) as mean_hr,
        APPROX_QUANTILES(heart_rate_value, 2)[OFFSET(1)] as median_hr,
        STDDEV(heart_rate_value) as std_hr,
        MIN(heart_rate_value) as min_hr,
        MAX(heart_rate_value) as max_hr,
        MAX(heart_rate_value) - MIN(heart_rate_value) as range_hr,
        COUNT(heart_rate_value) as count_hr,
        SUM(CASE WHEN heart_rate_value < 60 THEN 1 ELSE 0 END) / COUNT(heart_rate_value) as proportion_resting,
        SUM(CASE WHEN heart_rate_value BETWEEN 60 AND 100 THEN 1 ELSE 0 END) / COUNT(heart_rate_value) as proportion_moderate,
        SUM(CASE WHEN heart_rate_value > 100 THEN 1 ELSE 0 END) / COUNT(heart_rate_value) as proportion_high,
        AVG(CASE WHEN EXTRACT(HOUR FROM datetime) BETWEEN 6 AND 11 THEN heart_rate_value ELSE NULL END) as morning_hr,
        AVG(CASE WHEN EXTRACT(HOUR FROM datetime) BETWEEN 12 AND 17 THEN heart_rate_value ELSE NULL END) as afternoon_hr,
        AVG(CASE WHEN EXTRACT(HOUR FROM datetime) BETWEEN 18 AND 23 THEN heart_rate_value ELSE NULL END) as evening_hr,
        AVG(CASE WHEN EXTRACT(HOUR FROM datetime) BETWEEN 0 AND 5 THEN heart_rate_value ELSE NULL END) as night_hr
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.heart_rate_minute_level` heart_rate_minute_level
    WHERE
        heart_rate_minute_level.PERSON_ID IN (SELECT 
                DISTINCT person_id
            FROM 
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person
            WHERE 
                cb_search_person.person_id IN (SELECT 
                    person_id
                FROM 
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p
                WHERE 
                    has_fitbit = 1 )"""


### Base SQL template - Demographics

In [ ]:
# Base SQL template - Demographics
sql_template_demographics = """
    SELECT
        person.person_id,
        person.gender_concept_id,
        p_gender_concept.concept_name as gender,
        person.birth_datetime as date_of_birth,
        person.race_concept_id,
        p_race_concept.concept_name as race,
        person.ethnicity_concept_id,
        p_ethnicity_concept.concept_name as ethnicity,
        person.sex_at_birth_concept_id,
        p_sex_at_birth_concept.concept_name as sex_at_birth 
    FROM
        `""" + os.environ["WORKSPACE_CDR"] + """.person` person 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_gender_concept 
            ON person.gender_concept_id = p_gender_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_race_concept 
            ON person.race_concept_id = p_race_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_ethnicity_concept 
            ON person.ethnicity_concept_id = p_ethnicity_concept.concept_id 
    LEFT JOIN
        `""" + os.environ["WORKSPACE_CDR"] + """.concept` p_sex_at_birth_concept 
            ON person.sex_at_birth_concept_id = p_sex_at_birth_concept.concept_id  
    WHERE
        person.PERSON_ID IN (SELECT
                distinct person_id
            FROM 
                `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` cb_search_person
            WHERE 
                cb_search_person.person_id IN (SELECT 
                    person_id
                FROM 
                    `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_person` p
                WHERE 
                    has_fitbit = 1 )"""


### Add conditions for specific cohorts

In [ ]:
# Add conditions for specific cohorts
def get_sql_for_cohort_demographics(cohort, concept_ids):
    in_or_not = """
                AND cb_search_person.person_id IN ("""
    if cohort.startswith('not_'):
        in_or_not = """
                AND cb_search_person.person_id NOT IN ("""

    if isinstance(concept_ids, list):
        concept_condition = ", ".join([f"{cid}" for cid in concept_ids])
    else:
        concept_condition = f"{concept_ids}"
        
    return sql_template_demographics + in_or_not + f"""SELECT 
                    criteria.person_id
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + f""".cb_criteria` cr       
                            WHERE
                                concept_id IN ({concept_condition})       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1) 
                        AND is_standard = 1 )) criteria ) )"""



In [ ]:
# Add conditions for specific cohorts
def get_sql_for_cohort_hr(cohort, concept_ids):
    in_or_not = """
                AND cb_search_person.person_id IN ("""
    if cohort.startswith('not_'):
        in_or_not = """
                AND cb_search_person.person_id NOT IN ("""

    if isinstance(concept_ids, list):
        concept_condition = ", ".join([f"{cid}" for cid in concept_ids])
    else:
        concept_condition = f"{concept_ids}"
        
    return sql_template_hr + in_or_not + f"""SELECT 
                    criteria.person_id
                FROM
                    (SELECT
                        DISTINCT person_id, entry_date, concept_id 
                    FROM
                        `""" + os.environ["WORKSPACE_CDR"] + """.cb_search_all_events` 
                    WHERE
                        (concept_id IN(SELECT
                            DISTINCT c.concept_id 
                        FROM
                            `""" + os.environ["WORKSPACE_CDR"] + """.cb_criteria` c 
                        JOIN
                            (SELECT
                                CAST(cr.id as string) AS id       
                            FROM
                                `""" + os.environ["WORKSPACE_CDR"] + f""".cb_criteria` cr       
                            WHERE
                                concept_id IN ({concept_condition})       
                                AND full_text LIKE '%_rank1]%'      ) a 
                                ON (c.path LIKE CONCAT('%.', a.id, '.%') 
                                OR c.path LIKE CONCAT('%.', a.id) 
                                OR c.path LIKE CONCAT(a.id, '.%') 
                                OR c.path = a.id) 
                        WHERE
                            is_standard = 1 
                            AND is_selectable = 1) 
                        AND is_standard = 1 )) criteria ) )
    GROUP BY
        person_id,
        date"""



### Generate SQL for each cohort

In [ ]:
# Generate SQL for each cohort
queries_hr = \
    {cohort: get_sql_for_cohort_hr(cohort, concept_ids) for cohort, concept_ids in cohorts.items()}

queries_demographics = \
    {cohort: get_sql_for_cohort_demographics(cohort, concept_ids) for cohort, concept_ids in cohorts.items()}

### Review SQL for each cohort

In [ ]:
for q in queries_hr.keys():
    print()
    print(q)
    print()
    print('HR:')
    print(queries_hr[q])
    print()
    print('demographics:')
    print(queries_demographics[q])
    print()

## Retrieve Data from GBQ

In [ ]:
# Function to retrieve data from GBQ
def load_data_from_gbq(sql_query):
    return pd.read_gbq(
        sql_query,
        dialect="standard",
        use_bqstorage_api=("BIGQUERY_STORAGE_API_ENABLED" in os.environ),
        progress_bar_type="tqdm_notebook"
    )

In [ ]:
# Load data for each cohort
data_hr = {cohort: load_data_from_gbq(sql) for cohort, sql in queries_hr.items()}

In [ ]:
# Load data for each cohort
data_demo = {cohort: load_data_from_gbq(sql) for cohort, sql in queries_demographics.items()}

In [ ]:
df_anx_hr = data_hr['anxiety']
df_anx_demo = data_demo['anxiety']
df_dep_hr = data_hr['depression']
df_dep_demo = data_demo['depression']
df_no_anx_dep_hr = data_hr['not_anxiety_depression']
df_no_anx_dep_demo = data_demo['not_anxiety_depression']

In [ ]:
print(df_anx_demo.columns.values)

In [ ]:
print(df_anx_hr.columns.values)

In [ ]:
print(df_anx_hr.shape)
print(df_anx_demo.shape)
print(df_dep_hr.shape)
print(df_dep_demo.shape)
print(df_no_anx_dep_hr.shape)
print(df_no_anx_dep_demo.shape)

## Write DFs to file to save ~45min in future

In [ ]:
# Write Demographic and HR data to CSVs

dfs_names = [
    ('df_anx_hr', df_anx_hr),
    ('df_anx_demo', df_anx_demo),
    ('df_dep_hr', df_dep_hr),
    ('df_dep_demo', df_dep_demo),
    ('df_no_anx_dep_hr', df_no_anx_dep_hr),
    ('df_no_anx_dep_demo', df_no_anx_dep_demo)
]

for df_name, df in dfs_names:
    df.to_csv(f"{my_bucket}/data/dfs/{df_name}.csv")

---
---

### Above adds new HR data aggregated from `minute_level`  

___per `daily_summary` in v7 of the dataset is broken (only 1 zone/person/day)___

(so removed below)

---
---

In [ ]:
dfs_names = ['dataset_22083694_fitbit_activity_df',
#              'dataset_22083694_fitbit_heart_rate_summary_df',
             'dataset_22083694_fitbit_sleep_daily_summary_df',
             'dataset_29237714_fitbit_activity_df',
#              'dataset_29237714_fitbit_heart_rate_summary_df',
             'dataset_29237714_fitbit_sleep_daily_summary_df',
             'dataset_78185922_fitbit_activity_df',
#              'dataset_78185922_fitbit_heart_rate_summary_df',
             'dataset_78185922_fitbit_sleep_daily_summary_df'
            ]

In [ ]:
dfs = [dataset_22083694_fitbit_activity_df,
#        dataset_22083694_fitbit_heart_rate_summary_df,
       dataset_22083694_fitbit_sleep_daily_summary_df,
       dataset_29237714_fitbit_activity_df,
#        dataset_29237714_fitbit_heart_rate_summary_df,
       dataset_29237714_fitbit_sleep_daily_summary_df,
       dataset_78185922_fitbit_activity_df,
#        dataset_78185922_fitbit_heart_rate_summary_df,
       dataset_78185922_fitbit_sleep_daily_summary_df
      ]

In [ ]:
for df, df_name in zip(dfs, dfs_names):
    df.to_csv(f"{my_bucket}/data/dfs/{df_name}.csv")

## Read DFs from CSV

In [ ]:
RAM_start()

In [ ]:
# List objects in the bucket
print(subprocess.check_output(f"gsutil ls -r {my_bucket}", shell=True).decode('utf-8'))

In [ ]:
df_anx_hr = pd.read_csv(f"{my_bucket}/data/dfs/df_anx_hr.csv", index_col=0)
df_anx_demo = pd.read_csv(f"{my_bucket}/data/dfs/df_anx_demo.csv", index_col=0)
df_dep_hr = pd.read_csv(f"{my_bucket}/data/dfs/df_dep_hr.csv", index_col=0)
df_dep_demo = pd.read_csv(f"{my_bucket}/data/dfs/df_dep_demo.csv", index_col=0)
df_no_anx_dep_hr = pd.read_csv(f"{my_bucket}/data/dfs/df_no_anx_dep_hr.csv", index_col=0)
df_no_anx_dep_demo = pd.read_csv(f"{my_bucket}/data/dfs/df_no_anx_dep_demo.csv", index_col=0)

In [ ]:
dataset_22083694_fitbit_activity_df = pd.read_csv(f"{my_bucket}/data/dfs/dataset_22083694_fitbit_activity_df.csv", index_col=0)
# dataset_22083694_fitbit_heart_rate_summary_df = pd.read_csv(f"{my_bucket}/data/dfs/dataset_22083694_fitbit_heart_rate_summary_df.csv", index_col=0)
dataset_22083694_fitbit_sleep_daily_summary_df = pd.read_csv(f"{my_bucket}/data/dfs/dataset_22083694_fitbit_sleep_daily_summary_df.csv", index_col=0)
dataset_29237714_fitbit_activity_df = pd.read_csv(f"{my_bucket}/data/dfs/dataset_29237714_fitbit_activity_df.csv", index_col=0)
# dataset_29237714_fitbit_heart_rate_summary_df = pd.read_csv(f"{my_bucket}/data/dfs/dataset_29237714_fitbit_heart_rate_summary_df.csv", index_col=0)
dataset_29237714_fitbit_sleep_daily_summary_df = pd.read_csv(f"{my_bucket}/data/dfs/dataset_29237714_fitbit_sleep_daily_summary_df.csv", index_col=0)
dataset_78185922_fitbit_activity_df = pd.read_csv(f"{my_bucket}/data/dfs/dataset_78185922_fitbit_activity_df.csv", index_col=0)
# dataset_78185922_fitbit_heart_rate_summary_df = pd.read_csv(f"{my_bucket}/data/dfs/dataset_78185922_fitbit_heart_rate_summary_df.csv", index_col=0)
dataset_78185922_fitbit_sleep_daily_summary_df = pd.read_csv(f"{my_bucket}/data/dfs/dataset_78185922_fitbit_sleep_daily_summary_df.csv", index_col=0)

----
----

----
----

# Data Processing

In [ ]:
import numpy as np

#physical activity data
dep_act_df = dataset_78185922_fitbit_activity_df
anx_act_df = dataset_29237714_fitbit_activity_df
absent_act_df = dataset_22083694_fitbit_activity_df

#heart rate data
# dep_heart_df = dataset_78185922_fitbit_heart_rate_summary_df
# anx_heart_df = dataset_29237714_fitbit_heart_rate_summary_df
# absent_heart_df = dataset_22083694_fitbit_heart_rate_summary_df

#heart rate data
dep_heart_df = df_dep_hr
anx_heart_df = df_anx_hr
absent_heart_df = df_no_anx_dep_hr

#demographic data
dep_demo_df = df_dep_demo
anx_demo_df = df_anx_demo
absent_demo_df = df_no_anx_dep_demo

#sleep data
dep_sleep_df = dataset_78185922_fitbit_sleep_daily_summary_df
anx_sleep_df = dataset_29237714_fitbit_sleep_daily_summary_df
absent_sleep_df = dataset_22083694_fitbit_sleep_daily_summary_df

# anx_idx = np.unique(list(anx_sleep_df['person_id']) + list(anx_act_df['person_id']) + list(anx_heart_df['person_id']))
# anx_labels = pd.Series(1,index=anx_idx)
# dep_idx = np.unique(list(dep_sleep_df['person_id']) + list(dep_act_df['person_id']) + list(dep_heart_df['person_id']))
# dep_labels = pd.Series(1,index=dep_idx)
# absent_idx = np.unique(list(absent_sleep_df['person_id']) + list(absent_act_df['person_id']) + list(absent_heart_df['person_id']))
# absent_labels = pd.Series(1,index=absent_idx)

In [ ]:
print(f"intersect:\n{pd.Index(anx_sleep_df['person_id']).intersection(anx_act_df['person_id']).intersection(anx_demo_df['person_id']).intersection(anx_heart_df['person_id']).unique()}")
print()
print('vs')
print()
print(f"union:\n{pd.Index(anx_sleep_df['person_id']).union(anx_act_df['person_id']).union(anx_demo_df['person_id']).union(anx_heart_df['person_id']).unique()}")

__PER ABOVE__

## Choosing `intersection` per ensures individuals have data for all (sleep, activity, heart, demographic)

In [ ]:
# Extract unique person_ids and create labels
anx_idx = pd.Index(anx_sleep_df['person_id']).intersection(
                     anx_act_df['person_id']).intersection(
                     anx_demo_df['person_id']).intersection(
                     anx_heart_df['person_id']).unique()
anx_labels = pd.Series(1, index=anx_idx)

dep_idx = pd.Index(dep_sleep_df['person_id']).intersection(
                     dep_act_df['person_id']).intersection(
                     dep_demo_df['person_id']).intersection(
                     dep_heart_df['person_id']).unique()
dep_labels = pd.Series(1, index=dep_idx)

absent_idx = pd.Index(absent_sleep_df['person_id']).intersection(
                        absent_act_df['person_id']).intersection(
                        absent_demo_df['person_id']).intersection(
                        absent_heart_df['person_id']).unique()
absent_labels = pd.Series(1, index=absent_idx)


# # Extract unique person_ids and create labels
# anx_idx = pd.Index(anx_sleep_df['person_id']).union(anx_act_df['person_id']).union(anx_heart_df['person_id']).unique()
# anx_labels = pd.Series(1, index=anx_idx)

# dep_idx = pd.Index(dep_sleep_df['person_id']).union(dep_act_df['person_id']).union(dep_heart_df['person_id']).unique()
# dep_labels = pd.Series(1, index=dep_idx)

# absent_idx = pd.Index(absent_sleep_df['person_id']).union(absent_act_df['person_id']).union(absent_heart_df['person_id']).unique()
# absent_labels = pd.Series(1, index=absent_idx)

In [ ]:
labels = pd.concat([anx_labels,dep_labels,absent_labels],axis=1,join='outer')
labels = labels.fillna(0).astype(int).rename(columns={0:'Anxiety disorder',1:'Depressive disorder',2:'No disorder'})
print(labels)

In [ ]:
act_df = pd.concat([dep_act_df,anx_act_df,absent_act_df])
act_df = act_df.drop_duplicates(subset=['person_id', 'date'])

In [ ]:
heart_df = pd.concat([dep_heart_df,anx_heart_df,absent_heart_df])
heart_df = heart_df.drop_duplicates(subset=['person_id', 'date'])

In [ ]:
demo_df = pd.concat([dep_demo_df,anx_demo_df,absent_demo_df])
demo_df = demo_df.drop_duplicates(subset=['person_id'])

In [ ]:
demo_df.dtypes

In [ ]:
# Ensure the date_of_birth field is in datetime format
demo_df['date_of_birth'] = pd.to_datetime(demo_df['date_of_birth'])

# Function to calculate age
def calculate_age(born):
    today = datetime.now()
    return today.year - born.year - ((today.month, today.day) < (born.month, born.day))

# Apply the function to calculate age for each person
demo_df['age'] = demo_df['date_of_birth'].apply(calculate_age)
demo_df = demo_df.drop(columns=['date_of_birth'])
demo_df = demo_df.drop(columns=['gender_concept_id','race_concept_id','ethnicity_concept_id','sex_at_birth_concept_id'])
demo_df

In [ ]:
print(demo_df)

In [ ]:
demo_df.dtypes

### Demographic distributions

In [ ]:
for i in ['gender','race','ethnicity','sex_at_birth','age']:
    print()
    print(demo_df[i].value_counts())
    print()

----
----

**Later ('Scale and Split') exclusion of 'naps' would not work properly (per subset=[`person_id`,`sleep_date`] without considering `is_main_sleep`)**

In [ ]:
sleep_df = pd.concat([dep_sleep_df,anx_sleep_df,absent_sleep_df])
# sleep_df = sleep_df.drop_duplicates(subset=['person_id', 'sleep_date'])
sleep_df = sleep_df.drop_duplicates(subset=['person_id', 'sleep_date', 'is_main_sleep'])

# test_person_id = sleep_df['person_id'].unique()[5]

# test_person_sleep_df = sleep_df[sleep_df['person_id']==test_person_id]
# test_person_main_sleep_df = test_person_sleep_df[test_person_sleep_df['is_main_sleep']=='true']

# test_person_sleep_df.plot('sleep_date','minute_asleep',figsize=[20,5])
# test_person_main_sleep_df.plot('sleep_date','minute_asleep',figsize=[20,5])

### Sleep - `is_main_sleep` naps example

See example below

In [ ]:
temp = pd.concat([dep_sleep_df,anx_sleep_df,absent_sleep_df])

# Group by 'person_id' and 'date', then count the occurrences
grouped = temp.groupby(['person_id', 'sleep_date']).size().reset_index(name='count')

# Filter groups that have more than one row
mult_per_date = grouped[grouped['count'] > 1]
print(mult_per_date.head(3).tail(2))

In [ ]:
mult_dates = mult_per_date['sleep_date'].unique()

# Filter the temp DataFrame for the specified person_id and sleep_date in mult_dates
filtered_temp = temp[(temp['person_id'] == 1000107) & (temp['sleep_date'].isin(mult_dates))]

filtered_temp.sort_values('sleep_date').head(15).tail(4)

In [ ]:
variables_to_delete = ['temp', 'grouped', 'mult_per_date']

for var in variables_to_delete:
    try:
        # Attempt to delete each variable
        del globals()[var]
    except KeyError:
        # Handle the case where the variable does not exist
        print(f"Variable '{var}' does not exist and cannot be deleted.")


---
---

### HR Zone (currently N/A per source data flawed in v7)

In [ ]:
# RAM_start()

Takes a while, write to CSV below and read from instead of re-running (unless change something)

In [ ]:
# zone_names = ['Cardio', 'Out of Range', 'Fat Burn', 'Peak']
# zone_dfs = []

# # Iterate over each unique person_id
# for id in heart_df['person_id'].unique():
#     person_df = heart_df[heart_df['person_id'] == id]
    
#     # Initialize a DataFrame to store the person's data
#     person_zone_df = pd.DataFrame({'date': person_df['date'].unique()})
#     person_zone_df['person_id'] = id
    
#     # For each zone, pivot the minute_in_zone data
#     for zone in zone_names:
#         temp_df = person_df[person_df['zone_name'] == zone][['date', 'minute_in_zone']]
#         temp_df = temp_df.rename(columns={'minute_in_zone': zone})
#         person_zone_df = pd.merge(person_zone_df, temp_df, on='date', how='left')
    
#     # Append the person's DataFrame to the list
#     zone_dfs.append(person_zone_df)

# # Concatenate all individual person DataFrames
# zone_df = pd.concat(zone_dfs, ignore_index=True)

# # Fill NaN values with 0 (or appropriate value based on context)
# zone_df = zone_df.fillna(0)

# zone_df


# # Above uses ~10 GB below uses > 100 GB
# #
# # #Munge heart rate dataset so each zone is own column
# # zone_names = ['Cardio', 'Out of Range', 'Fat Burn', 'Peak']
# # dfs = []

# # for id in heart_df['person_id'].unique():
# #     person_df = heart_df[heart_df['person_id']==id]
# #     for zone in zone_names:
# #         temp_df = person_df[person_df['zone_name']==zone]
# #         temp_df = temp_df[['person_id','date','minute_in_zone']].rename(columns={'minute_in_zone':zone})
# #         dfs.append(temp_df)

# # zone_df = pd.concat(dfs,axis=1)



__Write CSV__

In [ ]:
# zone_df.to_csv(f"{my_bucket}/data/dfs/zone_df.csv")

In [ ]:
# zone_df.head(2)

__Read CSV__

In [ ]:
# zone_df = pd.read_csv(f"{my_bucket}/data/dfs/zone_df.csv", index_col=0)

In [ ]:
# zone_df.head(2)

----
---

Nobody has more than 1 non-zero value for a given date (see below)

In [ ]:
# # Define the columns to check
# columns_to_check = ['Cardio', 'Out of Range', 'Fat Burn', 'Peak']

# # Create a boolean DataFrame where each entry is True if the corresponding entry in the original DataFrame is non-zero
# non_zero = zone_df[columns_to_check] != 0

# # Sum across the columns for each row to count non-zero entries
# non_zero_counts = non_zero.sum(axis=1)

# # Check if any row has more than one non-zero entry
# rows_with_multiple_non_zero = zone_df[non_zero_counts > 1]

# # Display the rows with more than one non-zero value
# print(rows_with_multiple_non_zero)

In [ ]:
# RAM_stop()

### Merge df's

NOTE:  
  
Below merge will duplicate *_df rows for individuals who took a nap on a given date (`is_main_sleep` false, then true in another row)

In [ ]:
#Merge all dataframes into one
sleep_df = sleep_df.rename(columns={'sleep_date':'date'})

# daily_df = sleep_df.merge(act_df,on=['person_id','date'],how='inner').merge(heart_df,on=['person_id','date'],how='inner')
daily_df = sleep_df \
            .merge(act_df,on=['person_id','date'],how='inner') \
            .merge(heart_df,on=['person_id','date'],how='inner') ##\
            #.merge(demo_df,on=['person_id'],how='inner')        ## lots of duplication if done here
daily_df

# EDA

### __summarize `daily_df`__ (first 5,000)

In [ ]:
import os
import numpy as np
import pandas as pd
import pandas_profiling
import plotnine
from plotnine import *  # Provides a ggplot-like interface to matplotlib.
from IPython.display import display

## Plot setup.
theme_set(theme_bw(base_size = 11)) # Default theme for plots.

def get_boxplot_fun_data(df):
  """Returns a data frame with a y position and a label, for use annotating ggplot boxplots.

  Args:
    d: A data frame.
  Returns:
    A data frame with column y as max and column label as length.
  """
  d = {'y': max(df), 'label': f'N = {len(df)}'}
  return(pd.DataFrame(data=d, index=[0]))

# NOTE: if you get any errors from this cell, restart your kernel and run it again.


In [ ]:
# # Use snippet 'summarize_a_dataframe' to display summary statistics for a dataframe.
# # It assumes snippet 'Setup' has been executed.
# # See also https://towardsdatascience.com/exploring-your-data-with-just-1-line-of-python-4b35ce21a82d


# ## -----[ CHANGE THE DATAFRAME NAME(S) TO MATCH YOURS FROM DATASET BUILDER] -----
# daily_df.loc[:5000,:].profile_report()  # Examine up to the first 10,000 rows. Larger
#                                         # dataframes can be profiled, but it takes more time.


In [ ]:
# # Use snippet 'summarize_a_dataframe' to display summary statistics for a dataframe.
# # It assumes snippet 'Setup' has been executed.
# # See also https://towardsdatascience.com/exploring-your-data-with-just-1-line-of-python-4b35ce21a82d


# ## -----[ CHANGE THE DATAFRAME NAME(S) TO MATCH YOURS FROM DATASET BUILDER] -----
# demo_df.loc[:,:].profile_report()         # Examine up to the first 10,000 rows. Larger
#                                         # dataframes can be profiled, but it takes more time.


---
---

### Are there any people with more than one row per date?

In [ ]:
print('Total number of patients: '+str(len(np.unique(daily_df.person_id))))

In [ ]:
# Group by 'person_id' and 'date', then count the occurrences
grouped = daily_df.groupby(['person_id', 'date']).size().reset_index(name='count')

# Filter groups that have more than one row
print(grouped[grouped['count'] > 1])

del grouped

---

___Yes, but only 2 rows per date per naps (`is_main_sleep` True & False)___

---
---

# Write `daily_df` to CSV

---

(`daily_df_v2.csv`, `daily_df_labels_v2.csv`, `demographics_df.csv`)


---
---

In [ ]:
# daily_df.to_csv(f"{my_bucket}/data/dfs/daily_df.csv", index=False)
# labels.to_csv(f"{my_bucket}/data/dfs/daily_df_labels.csv")

# added HR data from `minute_level` source
daily_df.to_csv(f"daily_df_v2.csv", index=False)
labels.to_csv(f"daily_df_labels_v2.csv")

In [ ]:
# write corresponding Demographics to CSV
demo_df.to_csv(f"demographics_df.csv", index=False)

In [ ]:
!gsutil cp daily_df_v2.csv {my_bucket}/data/dfs/daily_df_v2.csv
!gsutil cp daily_df_labels_v2.csv {my_bucket}/data/dfs/daily_df_labels_v2.csv
!gsutil cp demographics_df.csv {my_bucket}/data/dfs/demographics_df.csv

---
---
---
---
---
---
---

---
---
---
---
---
---
---

---
---
---
---
---
---
---

In [ ]:
# List objects in the bucket
print(subprocess.check_output(f"gsutil ls {my_bucket}/data/dfs", shell=True).decode('utf-8'))